In [2]:
import torch
from torch import nn

c:\Users\marco\.venv\multimodal-ssl\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def MLP(dim, projection_size, widening_factor=4):
    
    hidden_size = dim * widening_factor
    
    return nn.Sequential(
        nn.Linear(dim, hidden_size),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_size, projection_size)
    )

def SimSiamMLP(dim, projection_size, widening_factor=4):
    
    hidden_size = dim * widening_factor
    
    return nn.Sequential(
        nn.Linear(dim, hidden_size, bias=False),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_size, hidden_size, bias=False),
        nn.BatchNorm1d(hidden_size),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_size, projection_size, bias=False),
        nn.BatchNorm1d(projection_size, affine=False)
    )

In [6]:
input = torch.randn(20, 128)

mlp = MLP(128, 128)
simsiam = SimSiamMLP(128, 128)

out_mlp = mlp(input)
print(out_mlp.shape)
out_sim = simsiam(input)
print(out_sim.shape)

torch.Size([20, 128])
torch.Size([20, 128])


In [19]:
def mean_of_last_hidden_states_1(
    last_hidden_state: torch.Tensor = None,
    ) -> torch.Tensor:
    
    batch_size, sequence_length, _ = last_hidden_state.size()
    attention_mask = torch.ones(batch_size, sequence_length)
    
    output_vectors = []
    
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(
        last_hidden_state.size()).float().to(last_hidden_state.device
    )
    sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
    
    sum_mask = input_mask_expanded.sum(1)
    sum_mask = torch.clamp(sum_mask, min=1e-9)
    
    output_vectors.append(sum_embeddings / sum_mask)
    output_vector = torch.cat(output_vectors, 0)
    
    return output_vector

def mean_of_last_hidden_states_2(
    last_hidden_state: torch.Tensor = None,
    ) -> torch.Tensor:
    
    x = [torch.layer_norm(tl.float(), tl.shape[-1:]) for tl in last_hidden_state]
    x = sum(x) / len(x)
    x = torch.layer_norm(x.float(), x.shape[-1:])
    
    return x

In [20]:
x = torch.randn(8, 20, 42, 128) # batch_size, sequence_length, hidden_size
y = [torch.randn(8, 20, 42, 128)] # batch_size, sequence_length, hidden_size

print(len(y))

x_1 = mean_of_last_hidden_states_1(x)
# y_1 = mean_of_last_hidden_states_1(y)

x_2 = mean_of_last_hidden_states_2(x)
y_2 = mean_of_last_hidden_states_2(y)

print("Outputs of mean pooling from projector head")
print(x_1.shape)
# print(y_1.shape)

print("\nOutputs of aggregation from from loss function")
print(x_2.shape)
print(y_2.shape)

loss_fn = nn.SmoothL1Loss(reduction='mean', beta=1.0)

# loss_1 = loss_fn(x_1, y_2)
loss_2 = loss_fn(x_2, y_2)

# print("\nLosses of mean pooling from projector head: ", loss_1)
print("Losses of aggregation from from loss function: ", loss_2)

1


ValueError: too many values to unpack (expected 3)